In [74]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy import stats
import numpy as np
import datetime

In [78]:
events = pd.read_csv('../games_data/games_data_all_seasons.csv')

In [90]:
def make_delta(entry):
    m, s = entry.split(':')
    return datetime.timedelta(minutes=int(m), seconds=int(s))
events['period_time'] = events['period_time'].apply(lambda entry: make_delta(entry))

AttributeError: 'Timedelta' object has no attribute 'split'

In [85]:
events[(events['period_type']=='OVERTIME')]['period_time'].dt.seconds

67         37
428        24
429        51
430        90
431        99
         ... 
387772    105
387773    118
387774    145
387775    146
387776    237
Name: period_time, Length: 6729, dtype: int64

In [19]:
def adjust_negative_value_x(row):
    if row['period_type'] == 'SHOOTOUT' or (row['team_side'] != 'left' and row['team_side'] != 'right'):
        row['coordinate_x'] = abs(row['coordinate_x'])
    elif row['team_side'] == 'right' and row['coordinate_x']<0:
        row['coordinate_x'] = abs(row['coordinate_x'])
    return row

In [101]:
rink_width = 100
rink_height = 42.5
events_new_coor = events.copy()[~(events['coordinate_x'].isnull()) & (~events['coordinate_y'].isnull())]
events_new_coor = events_new_coor.apply(lambda row: adjust_negative_value_x(row), axis=1)

In [102]:
events_new_coor['game_pk'] = events_new_coor['game_pk'].apply(lambda i: str(i))

In [105]:
events_2017=events_new_coor[events_new_coor['game_pk'].str.startswith('2017')]
events_2017_ott=events_new_coor[(events_new_coor['game_pk'].str.startswith('2017')) & (events_new_coor['team_id']=='PIT')]

In [113]:
times = events_2017_ott.groupby(['game_pk']).apply(lambda e: compute_time_played(e)).dt.seconds.sum()/3600
times

94.90694444444445

In [108]:
def compute_time_played(events):
    overtime = events[events['period_type']=='OVERTIME']
    if len(overtime)==0:
        return datetime.timedelta(minutes=60, seconds=0)
    periods=np.unique(overtime['period'])
    time = datetime.timedelta(minutes=60, seconds=0)
    for p in periods:
        if overtime[overtime['period']==p]['event_type'].isin(['GOAL']).any():
            time+=overtime[(overtime['period']==p) & (overtime['event_type']=='GOAL')]['period_time'].iloc[0]
        else:
            time+=datetime.timedelta(minutes=5, seconds=0)
    return time

In [48]:
kde_league=stats.gaussian_kde([events_2017['coordinate_x'],events_2017['coordinate_y']])
X, Y = np.mgrid[-rink_width:rink_width:200j, -rink_height:rink_height:100j]
positions = np.vstack([X.ravel(), Y.ravel()])
Z_league = np.reshape(kde_league(positions).T, X.shape)

In [49]:
kde_team=stats.gaussian_kde([events_2017_ott['coordinate_x'],events_2017_ott['coordinate_y']])
Z_team = np.reshape(kde_team(positions).T, X.shape)

## Histogram

In [57]:
xedges=np.linspace(-100,100,200)
yedges=np.linspace(-42.5,42.5,100)
hist_league, xedges, yedges = np.histogram2d(events_2017['coordinate_x'], events_2017['coordinate_y'], bins=(xedges, yedges))
hist_ott, xedges, yedges = np.histogram2d(events_2017_ott['coordinate_x'], events_2017_ott['coordinate_y'], bins=(xedges, yedges))

In [60]:
# Create figure
fig = go.Figure()

# Constants
img_width = 1100
img_height = 467
scale_factor = 0.75
rink_width = 100
rink_height = 42.5

# Add invisible scatter trace.
# This trace is added to help the autoresize logic work.
fig.add_trace(
    go.Scatter(
        x=[-rink_width, rink_width],
        y=[-rink_height, rink_height],
        mode="markers",
        marker_opacity=0
    )
)


fig.add_trace(
    go.Heatmap(z=np.rot90((hist_ott-hist_league)),
               x = X[:,0],
               y = Y[0,:],
               opacity=0.5
    )
)

# fig.add_vline(x=0, line_width=5)

fig.add_annotation(text="Offence",
                  xref="paper", yref="paper",
                  x=0.5, y=1, showarrow=False)

# fig.add_annotation(text="Defence",
#                   xref="paper", yref="paper",
#                   x=0.24, y=1, showarrow=False)

# Configure axes
fig.update_xaxes(
    visible=True,
    range=[-rink_width, rink_width],
    showgrid=False,
    showline=False
)

fig.update_yaxes(
    visible=True,
    range=[-rink_height, rink_height+2],
    # the scaleanchor attribute ensures that the aspect ratio stays constant
    scaleanchor="x",
    showgrid=False,
    showline=False
)

# Add image
fig.add_layout_image(
    dict(
        x=-rink_width,
        sizex= rink_width * 2,
        y=rink_height,
        sizey= rink_height * 2,
        xref="x",
        yref="y",
        opacity=1.0,
        layer="below",
        sizing="stretch",
        source="../../project/figures/nhl_rink.png")
)

# Configure other layout
fig.update_layout(
    width=img_width * scale_factor,
    height=img_height * scale_factor,
    margin={"l": 0, "r": 0, "t": 0, "b": 0},
)

# Disable the autosize on double click because it adds unwanted margins around the image
# More detail: https://plotly.com/python/configuration-options/
fig.show(config={'doubleClick': 'reset'})